In [3]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [4]:
# setting the bearer token in the environment variable to use for authentication later
# bearer token to be included in the empty quotes
os.environ['TOKEN'] = ''

In [5]:
# auth() that returns that bearer token stored in the environemnt 
def auth():
    return os.getenv('TOKEN')

In [6]:
# this function will take our bearer token, pass it for authorization and return headers we will use to access the API
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [7]:
# now that we can access the API, we will build the request for the endpoint we are going to use and the params we want to pass
def create_url(keyword, start_date, end_date, max_results = 500):
    
    search_url = "https://api.twitter.com/2/tweets/search/all" 

    # we can change parameters based on the requirement
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

In [8]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [9]:
def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that
        # 1. Author ID
        author_id = tweet['author_id']
        
        # 2. Tweet text
        tweet_text = tweet['text']

        # 3. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 4. Geolocation
        if ('geo' in tweet):   
            geo = tweet['geo']['place_id']
        else:
            geo = " "

        # 5. Tweet ID
        tweet_id = tweet['id']

        # 6. Language
        lang = tweet['lang']

        # 7. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']
        
        #8. retweeted_id and retweeted_type
        if 'referenced_tweets' in tweet:
            referenced_tweets_type = tweet['referenced_tweets'][0]['type']
            retweeted_id = tweet['referenced_tweets'][0]['id']
        else:
            referenced_tweets_type = " "
            retweeted_id = " "
            
            
        # 9. source
        source = tweet['source']

        
        
        # Assemble all data in a list
        res = [author_id,tweet_text,created_at,geo,tweet_id,lang,like_count,quote_count,reply_count,retweet_count,referenced_tweets_type,retweeted_id,source]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 


In [10]:
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "Vertiport lang:en"
start_list =    ['2015-01-01T00:00:00.000Z',
                 '2016-01-01T00:00:00.000Z',
                 '2017-01-01T00:00:00.000Z',
                 '2018-01-01T00:00:00.000Z',
                 '2019-01-01T00:00:00.000Z',
                 '2020-01-01T00:00:00.000Z',
                 '2021-01-01T00:00:00.000Z',
                 '2022-01-01T00:00:00.000Z'
                ]

end_list =      ['2015-12-31T00:00:00.000Z',
                 '2016-12-31T00:00:00.000Z',
                 '2017-12-31T00:00:00.000Z',
                 '2018-12-31T00:00:00.000Z',
                 '2019-12-31T00:00:00.000Z',
                 '2020-12-31T00:00:00.000Z',
                 '2021-12-31T00:00:00.000Z',
                 '2022-07-31T00:00:00.000Z'
                 ]
max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("Vertiport.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author_id','tweet','created_at','geo','tweet_id','lang','like_count','quote_count','reply_count','retweet_count','referenced_tweets_type','retweeted_id','source'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 500 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "Vertiport.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "Vertiport.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2015-01-01T00:00:00.000Z
# of Tweets added from this response:  325
Total # of Tweets added:  325
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2016-01-01T00:00:00.000Z
# of Tweets added from this response:  154
Total # of Tweets added:  479
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2017-01-01T00:00:00.000Z
# of Tweets added from this response:  380
Total # of Tweets added:  859
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  1jzu9lk96gu5npw44qf70phv54gfc74zb0m2qo17jw59
Start Date:  2018-01-01T00:00:00.000Z
# of Tweets added from this response:  499
Total # of Tweets added:  1358
-------------------
-------------------
Token:  1jzu9lk96gu5npw44qf70phv54gfc74zb0m2qo17jw59
Endpoint Response Code: 200
------------